In [15]:
from torch_snippets import *

In [16]:
import wget

In [5]:
output_directory = 'D:\datasets'
url = 'https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv'
wget.download(url, out=output_directory)

'D:\\datasets/train-annotations-object-segmentation.csv'

In [6]:
output_directory = 'D:\datasets'
url = 'https://raw.githubusercontent.com/openimages/dataset/master/dict.csv'
wget.download(url, out=output_directory)

'D:\\datasets/dict.csv'

In [17]:
# specify the classes we want our model to predict
required_classes = ['person', 'dog', 'bird', 'car', 'elephant', 'football', 'jug', 'laptop', 'Mushroom', 'Pizza', 'Rocket', 'Shirt', 'Traffic sign', 'Watermelon', 'Zebra']
required_classes = [c.lower() for c in required_classes]


In [18]:
import pandas as pd

In [19]:
classes = pd.read_csv('D:\datasets\classes.csv', header=None)
classes.columns = ['class', 'class_name']
classes = classes[classes['class_name'].map(lambda x: x in required_classes ) ]

In [20]:
classes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 43 to 7647
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   class       15 non-null     object
 1   class_name  15 non-null     object
dtypes: object(2)
memory usage: 360.0+ bytes


In [21]:
classes

,class,class_name
43,/m/01226z,football
224,/m/015p6,bird
601,/m/01c648,laptop
753,/m/01g317,person
1022,/m/01mqdt,traffic sign
1045,/m/01n4qj,shirt
2819,/m/02vx4,football
4675,/m/052sf,mushroom
5181,/m/0663v,pizza
5916,/m/0898b,zebra


In [22]:
from torch_snippets import *

In [4]:
# reading a large file in chunks and appending to a dataframe
def read_csv_chunks(path, chunksize=1000):
    df = pd.DataFrame()
    for chunk in pd.read_csv(path, chunksize=chunksize):
        df = df.append(chunk)
    return df

In [12]:
# df = pd.read_csv('D:/Datasets/annotations.csv', chunksize=1)
df = pd.read_csv('D:/Datasets/annotations.csv')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686666 entries, 0 to 2686665
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   MaskPath      object 
 1   ImageID       object 
 2   LabelName     object 
 3   BoxID         object 
 4   BoxXMin       float64
 5   BoxXMax       float64
 6   BoxYMin       float64
 7   BoxYMax       float64
 8   PredictedIoU  float64
 9   Clicks        object 
dtypes: float64(5), object(5)
memory usage: 205.0+ MB


In [23]:
data  = pd.merge(df, classes, left_on='LabelName', right_on='class')

In [24]:
# fetching fewer images for each class
subset_data = data.groupby('class_name').agg({'ImageID': lambda x: list(x)[:1000]})
subset_data = flatten(subset_data.ImageID.tolist())
subset_data = data[data['ImageID'].map(lambda x: x in subset_data)]
subset_masks = subset_data['MaskPath'].tolist()

In [27]:
subset_masks[1:10]

['18c1fd8342d55af8_m01g317_a9a97702.png',
 '42ea7facefea9a28_m01g317_333376e6.png',
 'bcb123bf7f256b1c_m01g317_a42b01bc.png',
 'a2bb5944e071223c_m01g317_7ef4ab8d.png',
 '4fd2304f1f65dc74_m01g317_336e67e1.png',
 '0113dbe2b087bd55_m01g317_d606a644.png',
 'cc8c1cec5a263473_m01g317_5dd13cd7.png',
 'd6dc04e65df11e78_m01g317_cfc9e8e5.png',
 'db35878a7d398f65_m01g317_d61cc87b.png']